In [ ]:
import pandas as pd
import requests
from io import StringIO

# Corrected URL (space encoded as %20)
url = "https://www.wildfoxindia.com/Kambalghar/invstocksch_invstock.csv"

try:
    # Add headers in case server blocks Python requests
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Read CSV into pandas DataFrame
    df = pd.read_csv(StringIO(response.text), low_memory=False)

    # Show first 5 rows
    print("✅ CSV loaded successfully. First 5 rows:\n")
    df.head()

except requests.exceptions.RequestException as e:
    print(f"❌ HTTP error while downloading: {e}")

except Exception as e:
    print(f"❌ Error reading CSV into DataFrame: {e}")


✅ CSV loaded successfully. First 5 rows:



In [ ]:
df.head(50)

,admsite_code,entdt,enttype,icode,loccode,qty
0,1,24-04-2024,STO,KG284052,5,-1.0
1,1,10-10-2023,GRC,KG134429,4,2.0
2,1,27-02-2024,STO,KG219061,2,-1.0
3,1,03-11-2024,STI,KG99981,4,1.0
4,1,29-03-2024,STO,KG283053,2,-3.0
5,1,03-12-2024,GRC,KG289896,4,1.0
6,1,09-11-2024,STO,KG389468,5,-1.0
7,1,19-08-2024,GRC,KG223458,4,1.0
8,1,23-02-2024,DCO,KG245044,4,-2.0
9,1,26-10-2024,STO,KG435590,5,-1.0


In [ ]:
df.shape

(7420214, 6)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7420214 entries, 0 to 7420213
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   admsite_code  int64  
 1   entdt         object 
 2   enttype       object 
 3   icode         object 
 4   loccode       int64  
 5   qty           float64
dtypes: float64(1), int64(2), object(3)
memory usage: 339.7+ MB


In [ ]:
import pandas as pd
import requests
from io import StringIO
from sqlalchemy import create_engine, text
import urllib
import pyodbc

# -------------------------------
# 1. DB credentials
# -------------------------------
DB_SERVER = "SWETHA\\SQLEXPRESS"
DB_DATABASE = "wildfox"
DB_USERNAME = "wildfox"
DB_PASSWORD = "wfx@123"
TABLE_NAME = "stock"

# CSV URL
url = "https://www.wildfoxindia.com/Kambalghar/invstocksch_invstock.csv"

# -------------------------------
# 2. Load CSV
# -------------------------------
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
response.raise_for_status()
df = pd.read_csv(StringIO(response.text), low_memory=False)

print("✅ CSV loaded. Shape:", df.shape)
print("📌 Columns:", df.columns.tolist())

# -------------------------------
# 3. Convert date column
# -------------------------------
df["entdt"] = pd.to_datetime(df["entdt"], format="%d-%m-%Y", errors="coerce")
print(f"📅 'entdt' converted to datetime, missing: {df['entdt'].isna().sum()}")

# -------------------------------
# 4. Fill missing values
# -------------------------------
df["qty"] = df["qty"].fillna(0)
df["admsite_code"] = df["admsite_code"].fillna(0).astype(int)
df["loccode"] = df["loccode"].fillna(0).astype(int)
df["enttype"] = df["enttype"].fillna("")
df["icode"] = df["icode"].fillna("")

# -------------------------------
# 5. Detect ODBC driver
# -------------------------------
drivers = [driver for driver in pyodbc.drivers() if "SQL Server" in driver]
if not drivers:
    raise Exception("❌ No ODBC SQL Server driver found. Install ODBC Driver 17 or 18.")
ODBC_DRIVER = drivers[-1]
print(f"✅ Using ODBC Driver: {ODBC_DRIVER}")

# -------------------------------
# 6. Connect to SQL Server
# -------------------------------
params = urllib.parse.quote_plus(
    f"DRIVER={{{ODBC_DRIVER}}};"
    f"SERVER={DB_SERVER};"
    f"DATABASE={DB_DATABASE};"
    f"UID={DB_USERNAME};"
    f"PWD={DB_PASSWORD};"
    f"TrustServerCertificate=Yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", fast_executemany=True)

# -------------------------------
# 7. Create table dynamically
# -------------------------------
create_table_query = f"""
IF OBJECT_ID('{TABLE_NAME}', 'U') IS NULL
CREATE TABLE {TABLE_NAME} (
    id BIGINT IDENTITY(1,1) PRIMARY KEY,
    admsite_code INT,
    entdt DATETIME,
    enttype NVARCHAR(50),
    icode NVARCHAR(100),
    loccode INT,
    qty DECIMAL(18,2)
);
"""
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    print(f"✅ Table '{TABLE_NAME}' created (if not exists)")

# -------------------------------
# 8. Insert data in safe chunks
# -------------------------------
chunk_size = 1000
for start in range(0, len(df), chunk_size):
    end = start + chunk_size
    df.iloc[start:end].to_sql(
        TABLE_NAME,
        con=engine,
        if_exists="append",
        index=False,
        method=None  # fast_executemany=True improves performance
    )
    print(f"Inserted rows {start} to {min(end, len(df))}")

print(f"🎉 Data inserted successfully into {DB_DATABASE}.{TABLE_NAME}")
